In [ ]:
from pathlib import Path
import re
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

root = Path('/Users/georgezhou/personal/research/serverless-jit/bench/test/wordcount/out')
cold_log = root / 'cold_dump.log'
warm_log = root / 'warm_load.log'

pat = re.compile(r'^\[(?P<sec>[0-9]+\.[0-9]+)s\]\[[^\]]*\]\[jit,compilation\]\s+(?P<rest>.*)$')


def parse_comp_levels(log_path: Path):
    times_lvl3 = []
    times_lvl4 = []
    if not log_path.exists():
        return times_lvl3, times_lvl4
    with log_path.open('r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            m = pat.match(line.strip())
            if not m:
                continue
            sec = float(m.group('sec'))
            rest = m.group('rest')
            # Tokenize after the tag; first token is compile id; the first numeric token after that is the level
            tokens = rest.split()
            if not tokens:
                continue
            # Skip compile id (first token)
            tokens_after_id = tokens[1:]
            level = None
            for t in tokens_after_id:
                if t.isdigit():
                    level = int(t)
                    break
            if level is None:
                continue
            if level == 3:
                times_lvl3.append(sec)
            elif level == 4:
                times_lvl4.append(sec)
    # Sort to ensure monotonic time
    times_lvl3.sort()
    times_lvl4.sort()
    return times_lvl3, times_lvl4

cold3, cold4 = parse_comp_levels(cold_log)
warm3, warm4 = parse_comp_levels(warm_log)

# Build cumulative series

def cumulative_xy(times):
    if not times:
        return [], []
    xs = []
    ys = []
    c = 0
    for t in times:
        c += 1
        xs.append(t)
        ys.append(c)
    return xs, ys

# Plot tier 3
fig, ax = plt.subplots(figsize=(8,4.5))
if cold3:
    x, y = cumulative_xy(cold3)
    ax.plot(x, y, label='cold (tier 3)', alpha=0.9)
if warm3:
    x, y = cumulative_xy(warm3)
    ax.plot(x, y, label='warm (tier 3)', alpha=0.9)
ax.set_title('Running count of tier 3 compilations')
ax.set_xlabel('Time (s)')
ax.set_ylabel('# tier 3 compilations')
ax.grid(True, alpha=0.3)
if cold3 or warm3:
    ax.legend()
else:
    print('[viz] No tier 3 events found in logs')
plt.tight_layout()
plt.show()

# Plot tier 4
fig, ax = plt.subplots(figsize=(8,4.5))
if cold4:
    x, y = cumulative_xy(cold4)
    ax.plot(x, y, label='cold (tier 4)', alpha=0.9)
if warm4:
    x, y = cumulative_xy(warm4)
    ax.plot(x, y, label='warm (tier 4)', alpha=0.9)
ax.set_title('Running count of tier 4 compilations')
ax.set_xlabel('Time (s)')
ax.set_ylabel('# tier 4 compilations')
ax.grid(True, alpha=0.3)
if cold4 or warm4:
    ax.legend()
else:
    print('[viz] No tier 4 events found in logs')
plt.tight_layout()
plt.show()
